In [1]:
class Opcode:
    VALUE = None
    N_PARAMS = None
    
    def run_op(self, intcode, *parameters):
        raise NotImplemented
        
    def __repr__(self):
        return f'{self.__class__.__name__}(value={self.VALUE}, n_params={self.N_PARAMS})'
        
        
class OP_ADD(Opcode):
    VALUE = '01'
    N_PARAMS = 3
        
    @staticmethod
    def run_op(intcode, x, y, write_address):
        x.resolve_value(), y.resolve_value()
        result = x.value + y.value
        
        intcode.write_value_to_address(result, write_address.value)
        
        
class OP_MUL(Opcode):
    VALUE = '02'
    N_PARAMS = 3
    
    @staticmethod
    def run_op(intcode, x, y, write_address):
        x.resolve_value(), y.resolve_value()
        
        result = x.value * y.value
        intcode.write_value_to_address(result, write_address.value)
        
        
        
class OP_IN(Opcode):
    VALUE = '03'
    N_PARAMS = 1
    
    @staticmethod
    def run_op(intcode, write_address):
        # for this OP, the address to write to is expected to always be the actual address value.
        # even if we are in positional mode, we do not lookup the value stored at the memory location!
        result = str(input())
        intcode.write_value_to_address(result, write_address.value)
        
        
        
class OP_OUT(Opcode):
    VALUE = '04'
    N_PARAMS = 1
    
    @staticmethod
    def run_op(intcode, write_address):
        write_address.resolve_value()
        
        print(write_address.value)
        
        
class OP_HALT(Opcode):
    VALUE = '99'
    N_PARAMS = 0
    
    @staticmethod
    def run_op(intcode, *args):
        print(intcode)
    

    
OPCODES = {
    opcode.VALUE : opcode
    for opcode in [
        OP_ADD,
        OP_MUL,
        OP_IN,
        OP_OUT,
        OP_HALT,
    ]
}
    

In [2]:
 class Parameter:
    def __init__(self, value, mode, intcode):
        self.value = value
        self.mode = mode
        self.intcode = intcode
    
    def resolve_value(self):
        # when in postional mode ('0'), we need to lookup the final
        # value from the intcode memory
        if self.mode == '0':
            self.value = int(self.intcode.get_value_from_address(self.value))
    
    def __repr__(self):
        return f'{self.__class__.__name__}(value={self.value}, mode={self.mode})'
            

In [3]:
class Instruction:
    def __init__(self, intcode, opcode, parameters):
        self.intcode = intcode
        self.opcode = opcode
        self.parameters = parameters
        
    
    def execute_instruction(self):
        self.opcode.run_op(self.intcode, *self.parameters)
        self.intcode.increment_instruction_pointer(self.opcode.N_PARAMS+1)
        return self.intcode
    
    @classmethod
    def from_current_instruction_pointer(cls, intcode):
        
        def pad(inst, n=5):
            required_padding = max(n - len(inst), 0)
            return f'{"0" * required_padding}{inst}'
        
        instruction = intcode.get_value_at_current_address()
        instruction = pad(instruction)
        
        opcode_value  =  instruction[3:]
        opcode = OPCODES[opcode_value]
        
        modes = list(reversed(instruction[:3]))

        parameters = [
            Parameter(
                value = int(intcode.get_value_relative_to_current_pointer(i+1)),
                mode = modes[i],
                intcode = intcode,
            ) for i in range(opcode.N_PARAMS)
        ]
        
        return cls(intcode, opcode, parameters)
    
    
    def __repr__(self):
        return f'{self.__class__.__name__}(opcode={self.opcode}, parameters={self.parameters})'

In [4]:
from pathlib import Path

class Intcode:
    
    def __init__(self, code, instruction_pointer=0):
        self.code = code
        self.instruction_pointer = instruction_pointer
        
    @property
    def current_instruction(self):
        return Instruction.from_current_instruction_pointer(self)
    
    @property
    def current_opcode(self):
        return self.current_instruction.opcode.VALUE
    
    def run(self):
        while self.current_opcode != OP_HALT.VALUE:
            self.execute_current_instruction()
        return self.code
    
    def execute_current_instruction(self):
        self.current_instruction.execute_instruction()
        
    def increment_instruction_pointer(self, increment):
        self.instruction_pointer += increment
        
    def get_value_at_current_address(self):
        return self.code[self.instruction_pointer]
    
    def get_value_from_address(self, address):
        return self.code[address]
    
    def get_value_relative_to_current_pointer(self, address):
        return self.get_value_from_address(self.instruction_pointer + address)
    
    def write_value_to_address(self, value, address):
        self.code[int(address)] = str(value)
        
            
    @classmethod
    def from_textfile(cls, file):
        code = Path('input.txt').read_text().split(',')
        return cls(code=code, instruction_pointer=0)
    
    
    def __repr__(self):
        return f'{self.__class__.__name__}(code={self.code}, instruction_pointer={self.instruction_pointer})'

    

In [5]:
def run_tests():
    assert Intcode(code=['1','0','0','0','99']).run()                  == ['2','0','0','0','99']
    assert Intcode(code=['2','3','0','3','99']).run()                  == ['2','3','0','6','99']
    assert Intcode(code=['2','4','4','5','99','0']).run()              == ['2','4','4','5','99','9801']
    assert Intcode(code=['1','1','1','4','99','5','6','0','99']).run() == ['30','1','1','4','2','5','6','0','99']
    assert Intcode(code=['1002','4','3','4','33']).run()               == ['1002', '4', '3', '4', '99']
    assert Intcode(code=['104', '2', '99', '2']).run()                 == ['104', '2', '99', '2']
    
    return('success ٩(●ᴗ●)۶')

run_tests()

2


'success ٩(●ᴗ●)۶'

In [6]:
Intcode.from_textfile('input.txt').run()

1
0
0
0
0
0
0
0
0
0
12440243


['3',
 '225',
 '1',
 '225',
 '6',
 '6',
 '1101',
 '1',
 '238',
 '225',
 '104',
 '0',
 '1102',
 '45',
 '16',
 '225',
 '2',
 '65',
 '191',
 '224',
 '1001',
 '224',
 '-3172',
 '224',
 '4',
 '224',
 '102',
 '8',
 '223',
 '223',
 '1001',
 '224',
 '5',
 '224',
 '1',
 '223',
 '224',
 '223',
 '1102',
 '90',
 '55',
 '225',
 '101',
 '77',
 '143',
 '224',
 '101',
 '-127',
 '224',
 '224',
 '4',
 '224',
 '102',
 '8',
 '223',
 '223',
 '1001',
 '224',
 '7',
 '224',
 '1',
 '223',
 '224',
 '223',
 '1102',
 '52',
 '6',
 '225',
 '1101',
 '65',
 '90',
 '225',
 '1102',
 '75',
 '58',
 '225',
 '1102',
 '53',
 '17',
 '224',
 '1001',
 '224',
 '-901',
 '224',
 '4',
 '224',
 '1002',
 '223',
 '8',
 '223',
 '1001',
 '224',
 '3',
 '224',
 '1',
 '224',
 '223',
 '223',
 '1002',
 '69',
 '79',
 '224',
 '1001',
 '224',
 '-5135',
 '224',
 '4',
 '224',
 '1002',
 '223',
 '8',
 '223',
 '1001',
 '224',
 '5',
 '224',
 '1',
 '224',
 '223',
 '223',
 '102',
 '48',
 '40',
 '224',
 '1001',
 '224',
 '-2640',
 '224',
 '4',
 '224',
 